In [4]:
import numpy as np
import xarray as xr
import pandas as pd
import glob, os
import matplotlib.pyplot as plt

In [3]:
hostname = !hostname
assert "login" not in hostname[0]

In [14]:
data_path = '/ocean/projects/atm200007p/walrus/for_jerry/train/'
grid_path = '/ocean/projects/atm200007p/jlin96/neurips_proj/mooers_metrics/test_data/E3SM-MMF_ne4_grid-info.orig.nc'
norm_path = '/ocean/projects/atm200007p/jlin96/neurips_proj/mooers_metrics/norm_factors/'
mli_mean = xr.open_dataset(norm_path + 'mli_mean.nc')
mli_min = xr.open_dataset(norm_path + 'mli_min.nc')
mli_max = xr.open_dataset(norm_path + 'mli_max.nc')
mlo_scale = xr.open_dataset(norm_path + 'mlo_scale.nc')
ne4_grid_info = xr.open_dataset(grid_path)

# in/out variable lists
vars_mli = ['state_t','state_q0001','state_ps','pbuf_SOLIN', 'pbuf_LHFLX', 'pbuf_SHFLX']
vars_mlo = ['ptend_t','ptend_q0001','cam_out_NETSW','cam_out_FLWDS','cam_out_PRECSC','cam_out_PRECC','cam_out_SOLS','cam_out_SOLL','cam_out_SOLSD','cam_out_SOLLD']

stride_sample = 6
f_mli1 = glob.glob(data_path + '*/E3SM-MMF.mli.0008-0[23456789]-*-*.nc')
f_mli2 = glob.glob(data_path + '*/E3SM-MMF.mli.0008-1[012]-*-*.nc')
f_mli3 = glob.glob(data_path + '*/E3SM-MMF.mli.0009-01-*-*.nc')
f_mli_data = sorted([*f_mli1, *f_mli2, *f_mli3])
f_mli_data = f_mli_data[::stride_sample]

In [72]:
file_ex = f_mli_data[1]
ds_input = xr.open_dataset(file_ex, engine = 'netcdf4')
ds_input = ds_input[vars_mli]
ds_input = ds_input.merge(ne4_grid_info[['lat','lon']])

ds_target = xr.open_dataset(file_ex.replace('.mli.','.mlo.'), engine='netcdf4')
ds_target = ds_target.merge(ne4_grid_info[['lat','lon']])
ds_target['ptend_t'] = (ds_target['state_t'] - ds_input['state_t'])/1200 # T tendency [K/s]
ds_target['ptend_q0001'] = (ds_target['state_q0001'] - ds_input['state_q0001'])/1200 # Q tendency [kg/kg/s]
ds_target = ds_target[vars_mlo]
ds_target_scaled = ds_target*mlo_scale
ds_target_batched = ds_target_scaled.stack({'batch':{'ncol'}})
ds_target_batched = ds_target_batched.to_stacked_array("mlvar", sample_dims=["batch"], name='mlo')
npy_target = ds_target_batched.values

In [73]:
true_tend = ds_target_scaled['ptend_t'].values.transpose()
true_tend.shape

(384, 60)

In [75]:
true_tend

array([[-0.01455373, -0.02465764, -0.01880114, ..., -0.03721235,
        -0.0394231 , -0.04024667],
       [-0.01727485, -0.01488511, -0.01479675, ..., -0.03272807,
         0.00257168,  0.00412592],
       [-0.01138445, -0.05077357, -0.03093411, ..., -0.0046749 ,
         0.0053451 , -0.08395734],
       ...,
       [ 0.03257066,  0.01889151,  0.03359342, ...,  0.01421662,
        -0.06821475, -0.13132317],
       [ 0.0371164 , -0.00488371,  0.00280938, ..., -0.06565217,
        -0.07793909, -0.13272313],
       [ 0.0396563 , -0.00254501,  0.02494834, ..., -0.03168243,
        -0.0627007 , -0.14773947]])

In [94]:
np.sum(y_true[384:384+384,0:60] - true_tend)

3.435801428703615e-08

In [80]:
y_true.shape

(1681920, 128)

In [99]:
num_samples = y_true.shape[0]
heating = y_true[:,:60].reshape((int(num_samples/384), 384, 60))

In [88]:
reheating = heating.transpose((2,0,1))
reheating.shape

(60, 4380, 384)

In [87]:
heating.shape

(4380, 384, 60)

In [100]:
np.sum(true_tend - heating[1,:,:])

-3.435801428703615e-08

In [93]:
np.sum(true_tend - reheating[:,1,:].transpose())

-3.435801428703615e-08

# Sungduk's method

In [107]:
FN_MODEL_OUTPUT = {'MLP':  '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/001_backup_phase-7_retrained_models_step2_lot-147_trial_0027.best.h5.npy',
                   'RPN':  '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/rpn_pred_v1_stride6.npy',
                   'CNN':  '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/val_predict_cnn_reshaped_stride6_FINAL.npy',
                   # 'cVAE': './model_outputs/cvae_preds_bestcrps.h5',
                   'cVAE': '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/cvae.h5',
                   'HSR': '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/hsr_preds_bestcrps.h5',
                  }

            
# model name
# (model name is used for the output)
model_name = 'MLP'

# input of validation dataset (npy)
fn_x_true = '/ocean/projects/atm200007p/jlin96/neurips_proj/e3sm_train_npy/val_input_stride6.npy'

# true output of validation dataset (npy)
fn_y_true = '/ocean/projects/atm200007p/jlin96/neurips_proj/e3sm_train_npy/val_target_stride6.npy'

# Model predicted output of varlidation dataset (npy)
fn_y_pred = FN_MODEL_OUTPUT[model_name]

# model grid information (nc)
fn_grid = '/ocean/projects/atm200007p/jlin96/neurips_proj/ClimSim_release/grid_info/E3SM-MMF_ne4_grid-info.orig.nc'

# normalization scale factors (nc)
fn_mli_mean  = '/ocean/projects/atm200007p/jlin96/neurips_proj/ClimSim_release/norm_factors/mli_mean.nc'
fn_mli_min   = '/ocean/projects/atm200007p/jlin96/neurips_proj/ClimSim_release/norm_factors/mli_min.nc'
fn_mli_max   = '/ocean/projects/atm200007p/jlin96/neurips_proj/ClimSim_release/norm_factors/mli_max.nc'
fn_mlo_scale = '/ocean/projects/atm200007p/jlin96/neurips_proj/ClimSim_release/norm_factors/mlo_scale.nc'

# fn_save_output
fn_save_metrics = f'./metrics/{model_name}.metrics.csv'
fn_save_metrics_avg = f'./metrics/{model_name}.metrics.lev-avg.csv'

# physical constatns from (E3SM_ROOT/share/util/shr_const_mod.F90)
grav    = 9.80616    # acceleration of gravity ~ m/s^2
cp      = 1.00464e3  # specific heat of dry air   ~ J/kg/K
lv      = 2.501e6    # latent heat of evaporation ~ J/kg
lf      = 3.337e5    # latent heat of fusion      ~ J/kg
ls      = lv + lf    # latent heat of sublimation ~ J/kg
rho_air = 101325./ (6.02214e26*1.38065e-23/28.966) / 273.15 # density of dry air at STP  ~ kg/m^3
                                                            # ~ 1.2923182846924677
                                                            # SHR_CONST_PSTD/(SHR_CONST_RDAIR*SHR_CONST_TKFRZ)
                                                            # SHR_CONST_RDAIR   = SHR_CONST_RGAS/SHR_CONST_MWDAIR
                                                            # SHR_CONST_RGAS    = SHR_CONST_AVOGAD*SHR_CONST_BOLTZ
rho_h20 = 1.e3       # density of fresh water     ~ kg/m^ 3

vars_mlo_energy_conv = {'ptend_t':cp,
                        'ptend_q0001':lv,
                        'cam_out_NETSW':1.,
                        'cam_out_FLWDS':1.,
                        'cam_out_PRECSC':lv*rho_h20,
                        'cam_out_PRECC':lv*rho_h20,
                        'cam_out_SOLS':1.,
                        'cam_out_SOLL':1.,
                        'cam_out_SOLSD':1.,
                        'cam_out_SOLLD':1.
                       }

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

# set dimemsion names for xarray datasets
dim_name_level  = 'lev'
dim_name_sample = 'sample'

x_true_sungduk = np.load(fn_x_true).astype(np.float64)
y_true_sungduk = np.load(fn_y_true).astype(np.float64)

if fn_y_pred[-3:] == '.h5':
    y_pred = xr.open_dataset(fn_y_pred)['pred'].values
else:
    y_pred = np.load(fn_y_pred).astype(np.float64)
N_samples = y_pred.shape[0] 

# load norm/scale factors
mlo_scale = xr.open_dataset(fn_mlo_scale)
mli_mean  = xr.open_dataset(fn_mli_mean)
mli_min   = xr.open_dataset(fn_mli_min)
mli_max   = xr.open_dataset(fn_mli_max)

In [108]:
np.sum(y_true_sungduk - y_true)

0.0

In [109]:
# load grid information
ds_grid = xr.open_dataset(fn_grid) # has ncol:384
N_ncol = len(ds_grid['ncol']) # length of ncol dimension (nlat * nlon)

# make area-weights
ds_grid['area_wgt'] = ds_grid['area'] / ds_grid['area'].mean('ncol')

# map ds_grid's ncol dimension -> the N_samples dimension of npy-loayd arrays (e.g., y_pred)
to_xarray = {'area_wgt': (dim_name_sample,np.tile(ds_grid['area_wgt'], int(N_samples/len(ds_grid['ncol'])))),
            }
to_xarray = xr.Dataset(to_xarray)

# add nsample-mapped grid variables back to ds_grid
ds_grid = xr.merge([ds_grid  [['P0', 'hyai', 'hyam','hybi','hybm','lat','lon','area']],
                    to_xarray[['area_wgt']]])

# list of ML output variables
vars_mlo = ['ptend_t','ptend_q0001','cam_out_NETSW','cam_out_FLWDS','cam_out_PRECSC',
            'cam_out_PRECC','cam_out_SOLS','cam_out_SOLL','cam_out_SOLSD','cam_out_SOLLD'] # mlo mean ML output.

# length of each variable
vars_mlo_len = {'ptend_t':60,
                'ptend_q0001':60,
                'cam_out_NETSW':1,
                'cam_out_FLWDS':1,
                'cam_out_PRECSC':1,
                'cam_out_PRECC':1,
                'cam_out_SOLS':1,
                'cam_out_SOLL':1,
                'cam_out_SOLSD':1,
                'cam_out_SOLLD':1
               }

# map the length of dimension to the name of dimension
len_to_dim = {60:dim_name_level,
              N_samples: dim_name_sample}

# Here, we first construct a dictionary of {var name: (dimension name, array-like)},
# then, map the dictionary to an Xarray Dataset.
# (ref: https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html)

DS = {}

for kds in ['true', 'pred']:
    if kds=='true':
        work = y_true
    elif kds=='pred':
        work = y_pred

    # [1] Construct dictionary for xarray dataset
    #     format is key for variable name /
    #               value for a turple of (dimension names, data).
    to_xarray = {}
    for k, kvar in enumerate(vars_mlo):

        # length of variable (ie, number of levels)
        kvar_len = vars_mlo_len[kvar]

        # set dimensions of variable
        if kvar_len == 60:
            kvar_dims = (dim_name_sample, dim_name_level)
        elif kvar_len == 1:
            kvar_dims = dim_name_sample

        # set start and end indices of variable in the loaded numpy array
        # then, add 'kvar':(kvar_dims, <np_array>) to dictionary
        if k==0: ind1=0
        ind2 = ind1 + kvar_len

        # scaled output
        kvar_data = np.squeeze(work[:,ind1:ind2])
        # unscaled output
        kvar_data = kvar_data / mlo_scale[kvar].values

        to_xarray[kvar] = (kvar_dims, kvar_data)

        ind1 = ind2

    # [2] convert dict to xarray dataset
    DS[kds] = xr.Dataset(to_xarray)

    # [3] add surface pressure ('state_ps') from ml input
    # normalized ps
    state_ps =  xr.DataArray(x_true_sungduk[:,120], dims=('sample'), name='state_ps')
    # denormalized ps
    state_ps = state_ps * (mli_max['state_ps'] - mli_min['state_ps']) + mli_mean['state_ps']
    DS[kds]['state_ps'] = state_ps

    # [4] add grid information
    DS[kds] = xr.merge([DS[kds], ds_grid])

    # [5] add pressure thickness of each level, dp
    # FYI, in a hybrid sigma vertical coordinate system, pressure at level z is
    # P[x,z] = hyam[z]*P0 + hybm[z]*PS[x,z],
    # where, hyam and hybm are 
    tmp = DS[kds]['P0']*DS[kds]['hyai'] + DS[kds]['state_ps']*DS[kds]['hybi']
    tmp = tmp.isel(ilev=slice(1,61)).values - tmp.isel(ilev=slice(0,60)).values
    tmp = tmp.transpose()
    DS[kds]['dp'] = xr.DataArray(tmp, dims=('sample', 'lev'))

    # [6] break (sample) to (ncol,time)
    N_timestep = int(N_samples/N_ncol)
    dim_ncol     = np.arange(N_ncol)
    dim_timestep = np.arange(N_timestep)
    new_ind = pd.MultiIndex.from_product([dim_timestep, dim_ncol],
                                         names=['time', 'ncol'])
    DS[kds] = DS[kds].assign_coords(sample=new_ind).unstack('sample')

In [110]:
# load grid information
ds_grid = xr.open_dataset(fn_grid) # has ncol:384
N_ncol = len(ds_grid['ncol']) # length of ncol dimension (nlat * nlon)

# make area-weights
ds_grid['area_wgt'] = ds_grid['area'] / ds_grid['area'].mean('ncol')

# map ds_grid's ncol dimension -> the N_samples dimension of npy-loayd arrays (e.g., y_pred)
to_xarray = {'area_wgt': (dim_name_sample,np.tile(ds_grid['area_wgt'], int(N_samples/len(ds_grid['ncol'])))),
            }
to_xarray = xr.Dataset(to_xarray)

# add nsample-mapped grid variables back to ds_grid
ds_grid = xr.merge([ds_grid  [['P0', 'hyai', 'hyam','hybi','hybm','lat','lon','area']],
                    to_xarray[['area_wgt']]])

In [111]:
vars_mlo = ['ptend_t','ptend_q0001','cam_out_NETSW','cam_out_FLWDS','cam_out_PRECSC',
            'cam_out_PRECC','cam_out_SOLS','cam_out_SOLL','cam_out_SOLSD','cam_out_SOLLD'] # mlo mean ML output.

# length of each variable
vars_mlo_len = {'ptend_t':60,
                'ptend_q0001':60,
                'cam_out_NETSW':1,
                'cam_out_FLWDS':1,
                'cam_out_PRECSC':1,
                'cam_out_PRECC':1,
                'cam_out_SOLS':1,
                'cam_out_SOLL':1,
                'cam_out_SOLSD':1,
                'cam_out_SOLLD':1
               }

# map the length of dimension to the name of dimension
len_to_dim = {60:dim_name_level,
              N_samples: dim_name_sample}

In [112]:
# Here, we first construct a dictionary of {var name: (dimension name, array-like)},
# then, map the dictionary to an Xarray Dataset.
# (ref: https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html)

DS = {}

for kds in ['true', 'pred']:
    if kds=='true':
        work = y_true
    elif kds=='pred':
        work = y_pred

    # [1] Construct dictionary for xarray dataset
    #     format is key for variable name /
    #               value for a turple of (dimension names, data).
    to_xarray = {}
    for k, kvar in enumerate(vars_mlo):

        # length of variable (ie, number of levels)
        kvar_len = vars_mlo_len[kvar]

        # set dimensions of variable
        if kvar_len == 60:
            kvar_dims = (dim_name_sample, dim_name_level)
        elif kvar_len == 1:
            kvar_dims = dim_name_sample

        # set start and end indices of variable in the loaded numpy array
        # then, add 'kvar':(kvar_dims, <np_array>) to dictionary
        if k==0: ind1=0
        ind2 = ind1 + kvar_len

        # scaled output
        kvar_data = np.squeeze(work[:,ind1:ind2])
        # unscaled output
        kvar_data = kvar_data / mlo_scale[kvar].values

        to_xarray[kvar] = (kvar_dims, kvar_data)

        ind1 = ind2

    # [2] convert dict to xarray dataset
    DS[kds] = xr.Dataset(to_xarray)

    # [3] add surface pressure ('state_ps') from ml input
    # normalized ps
    state_ps =  xr.DataArray(x_true_sungduk[:,120], dims=('sample'), name='state_ps')
    # denormalized ps
    state_ps = state_ps * (mli_max['state_ps'] - mli_min['state_ps']) + mli_mean['state_ps']
    DS[kds]['state_ps'] = state_ps

    # [4] add grid information
    DS[kds] = xr.merge([DS[kds], ds_grid])

    # [5] add pressure thickness of each level, dp
    # FYI, in a hybrid sigma vertical coordinate system, pressure at level z is
    # P[x,z] = hyam[z]*P0 + hybm[z]*PS[x,z],
    # where, hyam and hybm are 
    tmp = DS[kds]['P0']*DS[kds]['hyai'] + DS[kds]['state_ps']*DS[kds]['hybi']
    tmp = tmp.isel(ilev=slice(1,61)).values - tmp.isel(ilev=slice(0,60)).values
    tmp = tmp.transpose()
    DS[kds]['dp'] = xr.DataArray(tmp, dims=('sample', 'lev'))

    # [6] break (sample) to (ncol,time)
    N_timestep = int(N_samples/N_ncol)
    dim_ncol     = np.arange(N_ncol)
    dim_timestep = np.arange(N_timestep)
    new_ind = pd.MultiIndex.from_product([dim_timestep, dim_ncol],
                                         names=['time', 'ncol'])
    DS[kds] = DS[kds].assign_coords(sample=new_ind).unstack('sample')

In [115]:
sungduk_reshaped = DS['true']['ptend_t']

In [116]:
sungduk_reshaped.shape

(60, 4380, 384)

In [124]:
true_tend

array([[-0.01455373, -0.02465764, -0.01880114, ..., -0.03721235,
        -0.0394231 , -0.04024667],
       [-0.01727485, -0.01488511, -0.01479675, ..., -0.03272807,
         0.00257168,  0.00412592],
       [-0.01138445, -0.05077357, -0.03093411, ..., -0.0046749 ,
         0.0053451 , -0.08395734],
       ...,
       [ 0.03257066,  0.01889151,  0.03359342, ...,  0.01421662,
        -0.06821475, -0.13132317],
       [ 0.0371164 , -0.00488371,  0.00280938, ..., -0.06565217,
        -0.07793909, -0.13272313],
       [ 0.0396563 , -0.00254501,  0.02494834, ..., -0.03168243,
        -0.0627007 , -0.14773947]])

In [133]:
y_true_sungduk.shape

(1681920, 128)

In [135]:
y_true_sungduk[384:384+384,0:60]

array([[-0.01455373, -0.02465764, -0.01880115, ..., -0.03721235,
        -0.0394231 , -0.04024667],
       [-0.01727485, -0.01488511, -0.01479675, ..., -0.03272807,
         0.00257168,  0.00412592],
       [-0.01138445, -0.05077357, -0.03093411, ..., -0.0046749 ,
         0.0053451 , -0.08395734],
       ...,
       [ 0.03257066,  0.01889151,  0.03359342, ...,  0.01421662,
        -0.06821474, -0.13132317],
       [ 0.0371164 , -0.00488371,  0.00280938, ..., -0.06565217,
        -0.07793909, -0.13272312],
       [ 0.0396563 , -0.00254501,  0.02494834, ..., -0.03168243,
        -0.0627007 , -0.14773947]])

In [134]:
y_true[384:384+384,0:60]

array([[-0.01455373, -0.02465764, -0.01880115, ..., -0.03721235,
        -0.0394231 , -0.04024667],
       [-0.01727485, -0.01488511, -0.01479675, ..., -0.03272807,
         0.00257168,  0.00412592],
       [-0.01138445, -0.05077357, -0.03093411, ..., -0.0046749 ,
         0.0053451 , -0.08395734],
       ...,
       [ 0.03257066,  0.01889151,  0.03359342, ...,  0.01421662,
        -0.06821474, -0.13132317],
       [ 0.0371164 , -0.00488371,  0.00280938, ..., -0.06565217,
        -0.07793909, -0.13272312],
       [ 0.0396563 , -0.00254501,  0.02494834, ..., -0.03168243,
        -0.0627007 , -0.14773947]])

In [126]:
sungduk_reshaped[:,1,:]

<xarray.DataArray 'ptend_t' (lev: 60, ncol: 384)>
array([[-1.44865167e-05, -1.71950696e-05, -1.13318691e-05, ...,
         3.24202265e-05,  3.69449725e-05,  3.94731410e-05],
       [-2.45437539e-05, -1.48163629e-05, -5.05390672e-05, ...,
         1.88042616e-05, -4.86115686e-06, -2.53325367e-06],
       [-1.87143109e-05, -1.47284099e-05, -3.07912408e-05, ...,
         3.34382664e-05,  2.79640695e-06,  2.48331180e-05],
       ...,
       [-3.70404853e-05, -3.25769117e-05, -4.65330665e-06, ...,
         1.41509591e-05, -6.53489502e-05, -3.15361042e-05],
       [-3.92410186e-05,  2.55979893e-06,  5.32041150e-06, ...,
        -6.78996898e-05, -7.75791260e-05, -6.24111087e-05],
       [-4.00607872e-05,  4.10686010e-06, -8.35695814e-05, ...,
        -1.30716648e-04, -1.32110132e-04, -1.47057125e-04]])
Coordinates:
    time     int64 1
  * ncol     (ncol) int64 0 1 2 3 4 5 6 7 8 ... 376 377 378 379 380 381 382 383
  * lev      (lev) float64 0.07835 0.1411 0.2529 0.4493 ... 963.6 978.0 992.6

In [131]:
ds_target['ptend_t'].values

array([[-1.44865171e-05, -1.71950690e-05, -1.13318695e-05, ...,
         3.24202264e-05,  3.69449728e-05,  3.94731406e-05],
       [-2.45437545e-05, -1.48163624e-05, -5.05390690e-05, ...,
         1.88042622e-05, -4.86115670e-06, -2.53325366e-06],
       [-1.87143101e-05, -1.47284095e-05, -3.07912408e-05, ...,
         3.34382649e-05,  2.79640701e-06,  2.48331175e-05],
       ...,
       [-3.70404845e-05, -3.25769124e-05, -4.65330649e-06, ...,
         1.41509593e-05, -6.53489532e-05, -3.15361053e-05],
       [-3.92410204e-05,  2.55979898e-06,  5.32041133e-06, ...,
        -6.78996910e-05, -7.75791252e-05, -6.24111107e-05],
       [-4.00607876e-05,  4.10685994e-06, -8.35695811e-05, ...,
        -1.30716641e-04, -1.32110137e-04, -1.47057126e-04]])

In [127]:
ds_target_scaled['ptend_t'].values

array([[-0.01455373, -0.01727485, -0.01138445, ...,  0.03257066,
         0.0371164 ,  0.0396563 ],
       [-0.02465764, -0.01488511, -0.05077357, ...,  0.01889151,
        -0.00488371, -0.00254501],
       [-0.01880114, -0.01479675, -0.03093411, ...,  0.03359342,
         0.00280938,  0.02494834],
       ...,
       [-0.03721235, -0.03272807, -0.0046749 , ...,  0.01421662,
        -0.06565217, -0.03168243],
       [-0.0394231 ,  0.00257168,  0.0053451 , ..., -0.06821475,
        -0.07793909, -0.0627007 ],
       [-0.04024667,  0.00412592, -0.08395734, ..., -0.13132317,
        -0.13272313, -0.14773947]])

In [130]:
ds_target['ptend_t'].values

array([[-1.44865171e-05, -1.71950690e-05, -1.13318695e-05, ...,
         3.24202264e-05,  3.69449728e-05,  3.94731406e-05],
       [-2.45437545e-05, -1.48163624e-05, -5.05390690e-05, ...,
         1.88042622e-05, -4.86115670e-06, -2.53325366e-06],
       [-1.87143101e-05, -1.47284095e-05, -3.07912408e-05, ...,
         3.34382649e-05,  2.79640701e-06,  2.48331175e-05],
       ...,
       [-3.70404845e-05, -3.25769124e-05, -4.65330649e-06, ...,
         1.41509593e-05, -6.53489532e-05, -3.15361053e-05],
       [-3.92410204e-05,  2.55979898e-06,  5.32041133e-06, ...,
        -6.78996910e-05, -7.75791252e-05, -6.24111107e-05],
       [-4.00607876e-05,  4.10685994e-06, -8.35695811e-05, ...,
        -1.30716641e-04, -1.32110137e-04, -1.47057126e-04]])

In [57]:
flat_tend = y_true[:,0:60]

In [71]:
np.sum(true_tend - flat_tend[384:384+384, :])

8.175422371833462e-08

In [52]:
np.reshape(y_true, ())

(1681920, 128)

In [7]:
fn_y_true = '/ocean/projects/atm200007p/jlin96/neurips_proj/e3sm_train_npy/val_target_stride6.npy'
y_true = np.load(fn_y_true).astype(np.float64)

In [8]:
y_true.shape

(1681920, 128)

In [42]:
ds_target

<xarray.Dataset>
Dimensions:         (lev: 60, ncol: 384)
Dimensions without coordinates: lev, ncol
Data variables:
    ptend_t         (lev, ncol) float64 -1.36e-05 -1.551e-05 ... -0.0001436
    ptend_q0001     (lev, ncol) float64 0.0 0.0 0.0 ... 1.618e-08 1.89e-08
    cam_out_NETSW   (ncol) float64 ...
    cam_out_FLWDS   (ncol) float64 ...
    cam_out_PRECSC  (ncol) float64 ...
    cam_out_PRECC   (ncol) float64 ...
    cam_out_SOLS    (ncol) float64 ...
    cam_out_SOLL    (ncol) float64 ...
    cam_out_SOLSD   (ncol) float64 ...
    cam_out_SOLLD   (ncol) float64 ...
Attributes:
    ne:        4
    fv_nphys:  2
    calendar:  NO_LEAP

In [40]:
y_true[1,:]

array([-1.55859012e-02, -1.78654157e-02, -1.52773447e-02, -5.26887253e-02,
       -8.38829651e-02, -1.14158742e-01, -1.10444278e-01, -8.13113153e-02,
       -5.06888554e-02, -4.00450528e-02, -2.96386871e-02, -2.25470364e-02,
       -1.77620780e-02, -1.35050863e-02, -9.06716939e-03, -5.53473504e-03,
       -6.43568626e-03,  2.24380405e-03, -7.31676398e-03, -6.73984876e-03,
        3.59389931e-03,  6.68655382e-03, -1.79768018e-02, -2.21105143e-02,
        1.88034922e-02, -1.51251061e-02, -3.15982290e-02, -1.80596206e-02,
       -3.07185277e-02, -1.18403370e-02, -1.18410168e-02, -6.54459093e-03,
        5.46707120e-03, -3.02739423e-02, -3.24615687e-02, -3.76746766e-02,
       -2.21766196e-02, -3.13716009e-02, -1.09762829e-02, -2.85710469e-02,
       -1.80315431e-02, -2.63136905e-02, -3.50125059e-02, -3.91680822e-02,
       -2.39459295e-02, -3.83239985e-02, -3.47019732e-02, -3.25246826e-02,
       -2.38250252e-02, -3.05242892e-02,  1.78362429e-02, -2.73267226e-03,
       -2.62119831e-03, -

In [34]:
y_true[0,:]

array([-1.36620812e-02, -2.80409772e-02, -2.15656627e-02, -5.23943938e-02,
       -8.13384652e-02, -1.10813349e-01, -1.06115066e-01, -7.64378756e-02,
       -4.98877764e-02, -3.91743705e-02, -2.72510257e-02, -1.97552871e-02,
       -1.45448223e-02, -1.03805251e-02, -5.29604033e-03, -7.86766876e-04,
        1.00812223e-03,  1.03712187e-03,  2.60534324e-03,  6.22053689e-04,
       -9.80621320e-04, -4.21181973e-03, -8.36136844e-03, -1.23731298e-02,
       -1.74749307e-02, -2.14770287e-02, -2.19386593e-02, -2.27263030e-02,
       -2.29678582e-02, -2.25563366e-02, -2.36029029e-02, -2.46444885e-02,
       -2.34153941e-02, -2.36174222e-02, -2.38939375e-02, -2.45916806e-02,
       -2.31757555e-02, -2.32806299e-02, -2.33938787e-02, -2.34074965e-02,
       -2.29851063e-02, -2.21269783e-02, -2.35317610e-02, -2.45282371e-02,
       -2.89273020e-02, -2.49604452e-02, -2.53472887e-02, -2.64138561e-02,
       -2.98754536e-02, -2.96021532e-02, -3.14517878e-02, -3.01577393e-02,
       -2.94105019e-02, -

In [117]:
true_tend.shape

(384, 60)

In [125]:
ds_target_scaled['ptend_t'].values

array([[-0.01455373, -0.01727485, -0.01138445, ...,  0.03257066,
         0.0371164 ,  0.0396563 ],
       [-0.02465764, -0.01488511, -0.05077357, ...,  0.01889151,
        -0.00488371, -0.00254501],
       [-0.01880114, -0.01479675, -0.03093411, ...,  0.03359342,
         0.00280938,  0.02494834],
       ...,
       [-0.03721235, -0.03272807, -0.0046749 , ...,  0.01421662,
        -0.06565217, -0.03168243],
       [-0.0394231 ,  0.00257168,  0.0053451 , ..., -0.06821475,
        -0.07793909, -0.0627007 ],
       [-0.04024667,  0.00412592, -0.08395734, ..., -0.13132317,
        -0.13272313, -0.14773947]])